In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg') 

# Use Agg backend for canvas
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

from python_code.estimate_essential_matrix import estimateEssentialMatrix
from python_code.decompose_essential_matrix import decomposeEssentialMatrix
from python_code.disambiguate_relative_pose import disambiguateRelativePose
from python_code.linear_triangulation import linearTriangulation
from python_code.draw_camera import drawCamera

In [6]:
# inputs
img_1 = np.array(cv2.imread('./data/0001.jpg'))
img_2 = np.array(cv2.imread('./data/0002.jpg'))

K = np.array([  [1379.74,   0,          760.35],
                [    0,     1382.08,    503.41],
                [    0,     0,          1 ]] )

# Load outlier-free point correspondences
p1 = np.loadtxt('./data/matches0001.txt')
p2 = np.loadtxt('./data/matches0002.txt')

p1 = np.r_[p1, np.ones((1, p1.shape[1]))]
p2 = np.r_[p2, np.ones((1, p2.shape[1]))]

# Estimate the essential matrix E using the 8-point algorithm
E = estimateEssentialMatrix(p1, p2, K, K);
print("E:\n", E)

F:
 [[-9.57519791e-10 -5.24686714e-08  2.23308995e-05]
 [ 3.76386464e-07  3.96320048e-09  1.93948110e-03]
 [-1.94627696e-04 -2.23387780e-03  1.57076992e-01]]
E:
 [[-1.82281364e-03 -1.00053090e-01 -6.63710751e-03]
 [ 7.17735511e-01  7.57028808e-03  3.07880664e+00]
 [-8.11154198e-03 -3.13977789e+00  2.30496711e-03]]


In [9]:
# Extract the relative camera positions (R,T) from the essential matrix
# Obtain extrinsic parameters (R,t) from E
Rots, u3 = decomposeEssentialMatrix(E)

# Disambiguate among the four possible configurations
R_C2_W,T_C2_W = disambiguateRelativePose(Rots, u3, p1, p2, K, K)


# Triangulate a point cloud using the final transformation (R,T)
M1 = K @ np.eye(3,4)
M2 = K @ np.c_[R_C2_W, T_C2_W]
P = linearTriangulation(p1, p2, M1, M2)

# Visualize the 3-D scene
fig = plt.figure()
ax = fig.add_subplot(1, 3, 1, projection='3d')

# R,T should encode the pose of camera 2, such that M1 = [I|0] and M2=[R|t]

# P is a [4xN] matrix containing the triangulated point cloud (in
# homogeneous coordinates), given by the function linearTriangulation
ax.scatter(P[0,:], P[1,:], P[2,:], marker = 'o')

# Display camera pose
drawCamera(ax, np.zeros((3,)), np.eye(3), length_scale = 2)
ax.text(-0.1,-0.1,-0.1,"Cam 1")

center_cam2_W = -R_C2_W.T @ T_C2_W
drawCamera(ax, center_cam2_W, R_C2_W.T, length_scale = 2)
ax.text(center_cam2_W[0]-0.1, center_cam2_W[1]-0.1, center_cam2_W[2]-0.1,'Cam 2')

# Display matched points
ax = fig.add_subplot(1,3,2)
ax.imshow(img_1)
ax.scatter(p1[0,:], p1[1,:], color = 'y', marker='s')
ax.set_title("Image 1")

ax = fig.add_subplot(1,3,3)
ax.imshow(img_2)
ax.scatter(p2[0,:], p2[1,:], color = 'y', marker='s')
ax.set_title("Image 2")

plt.show()

/tmp/ipykernel_2065/3528173781.py:43: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Error in callback <function _draw_all_if_interactive at 0x7fa845653f70> (for post_execute):


AttributeError: 'RendererAgg' object has no attribute 'M'